# Apple State Identifier

Apple State Identifier is a convolutional neural network trained with pictures of good and rotten apples to predict how long a user's apple will be good for when stored in the refrigerator or in the pantry.

# Importing

Import the necessary packages to effectively run the Apple State Identifier.

In [1]:
import os
import urllib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import requests
import torch
import copy
import sklearn
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.neural_network
import PIL
from PIL import Image

ModuleNotFoundError: No module named 'requests'

# Load Data


Let us define first a function called _resize_images(localpath, subpaths)_ where _localpath_ is the main path to the directory containing all data for good and rotten apples, and where _subpaths_ is a list of all subdirectories inside _localpath_. This function will resize all images to a uniform value of "|insert-here|", and will convert any non-RGB pictures to RGB. 
    
After running this, two values will be returned: an X and a y, where X is a Numpy array containing the pictures in RGB mode, and where y is the label to indicate whether a picture is for a good or rotten apple.

In [ ]:
# TODO: Define a default size and delete first for loop calculating average size.

def resize_images(localpath, subpaths):
    # to calc avg
    sizes_x = np.array([], dtype='float32')
    sizes_y = np.array([], dtype='float32')
    targets = np.array([], dtype='int64')
    
    num_pics = 0
    
    for subpath in subpaths:
        for file in os.listdir(localpath + subpath):
            
            if subpath == 'good/':
                targets = np.append(1, targets)
            elif subpath == 'bad/':
                targets = np.append(0, targets)
                
            f_img = localpath+subpath+file
            img = Image.open(f_img)
            
            if img.mode != 'RGB':
                img = img.convert('RGB')
                img.save(f_img)
                
            sizes_x = np.append(img.size[0], sizes_x)
            sizes_y = np.append(img.size[1], sizes_y)
            num_pics = num_pics + 1
    
    avg_x = int(np.average(sizes_x))
    avg_y = int(np.average(sizes_y))
    
    all_pics_resized = np.zeros((num_pics, avg_y, avg_x, 3), dtype='int64')
    
    num_pics = 0
    for subpath in subpaths:
        for file in os.listdir(localpath + subpath):
            f_img = localpath+subpath+file
            img = Image.open(f_img)
            img = img.resize((avg_x, avg_y))
#             print(img.size)
            all_pics_resized[num_pics] = img
            num_pics = num_pics + 1
    
    return all_pics_resized.reshape(num_pics, avg_y, avg_x, 3), targets

Call our function _resize_images(localpath, subdirectories)_ defined above to obtain the data for the convnet.

In [ ]:
# Main data directory
f = r'data/'

# Subdirectories
f_good_apples = r'good/'
f_bad_apples = r'bad/'

X, y = resize_images(f, [f_good_apples, f_bad_apples])

print(X.shape, y.shape)

This output right above tells us we have a total of 120 pictures (60 good, 60 rotten) of size |size| in mode RGB.

Now, let us print one of our images to confirm that everything is going as intended so far.

In [ ]:
plt.imshow(X[68]);

TODO: split training and testing data

# Train Convolutional Neural Network

For our convolutional neural network to take in data, we have to convert our _X_ and _y_ from the section above to PyTorch objects. Note that for _X_ we clip the data in the range \[0..1\] because this way |insert reason|

In [ ]:
X_torch, y_torch = torch.tensor((X - np.amin(X)) / (np.amax(X) - np.amin(X)), dtype=torch.float32), torch.from_numpy(y)
X_torch = X_torch.reshape(69, 3, 390, 453)


# Preprocess all the data first
# X_torch_all, y_torch_all = torch.tensor((X - np.amin(X)) / (np.amax(X) - np.amin(X)), dtype=torch.float32), torch.from_numpy(y)
# X_torch_all = X_torch_all.reshape(100, 3, 390, 453)

# Split into training and test
# X_train_torch, X_test_torch, y_train_torch, y_test_torch  = sklearn.model_selection.train_test_split(X_torch_all, y_torch_all, train_size=0.7, random_state=0)


print(X_torch.shape, y_torch.shape)

Next, the model we follow to create the Apple State Identifier neural network is the following:
- Feature learning
  - Layer 1
    - Convolutional 2D layer
    - ReLU activation
    - MaxPooling

  - Layer 2
    - Convolutional 2D layer
    - ReLU activation
    
- Classification
  - Flatten
  - Fully Connected (TODO: missing. Can be set up through loss)
  - Linear
  
  
The reason we have this setup is because |insert reason|

Moreover, our options are:
- Number of filters: 
- Filter size:
- Bias: 
- Padding: 
- Stride: 
  
Let us proceed to code this:    

In [ ]:
torch.manual_seed(0)

num_filters = 8
filter_size = 5

model = torch.nn.Sequential(
    torch.nn.Conv2d(3, num_filters, padding=filter_size//2, kernel_size=filter_size, stride=2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.Conv2d(8, num_filters, kernel_size=filter_size, stride=1),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(109 * 93 * num_filters, 2)
)

output = model(X_torch)
# output = model(X_train_torch)

print(output.shape)

TODO: does it make sense to have this below? The epoch run will do this same thing but hundreds of times

In [ ]:
y_pred = torch.softmax(model(X_torch[:69]), dim=1)
# y_pred = torch.softmax(model(X_test_torch[:15]), dim=1)
print(y_pred)

y_true = torch.zeros((69, 2))
y_true[torch.arange(69), y_torch[:69]] = 1
# y_true_test = torch.zeros((15, 2))
# y_true_test[torch.arange(15), y_test_torch[:15]] = 1
print(y_true)

In [ ]:
untrained_model = copy.deepcopy(model)

Now that we have the model we want, we give it our training data to train it. Since we have a binary classification problem (good or rotten), we use torch.nn.BCEWithLogitsLoss() which will make our model fully connected.

Also, let us we set a high epoch and print our training loss for each one to see how well the model is doing.

In [ ]:
model = copy.deepcopy(untrained_model)

# The number of times to evaluate the full training data (in this case, number of gradient steps)
num_epoch = 50

# Your code for defining loss, optimizer, and training loop here. Aim for 10-12 lines.

loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

for epoch in range(1, num_epoch+1):
    y_pred = model(X_torch)
    loss_value = loss(y_pred, y_true)
#     y_pred = model(X_train_torch)
#     loss_value = loss(y_pred, y_true_test)
    
    model.zero_grad()
    loss_value.backward()
    optimizer.step()
    
    if epoch == 1 or epoch % 10 == 0:
        print("Epoch %d had training loss %.4f" % (epoch, loss_value.item()))

# Testing

We now know we have a working network and what its loss is for multiple epoch runs, but what else could we look at?

Something extremely important is the overall accuracy of the model, and the stats for each category.

Below, for our testing set, we can see the label of each picture (left) and how it was classified (right).

In [2]:
y_true_index = torch.tensor([torch.argmax(y_true[i]) for i in range(len(y_true))])
# y_true_index = torch.tensor([torch.argmax(y_true_test[i]) for i in range(len(y_true_test))])
y_pred_index = torch.tensor([torch.argmax(y_pred[i]) for i in range(len(y_pred))])

plt.figure(figsize=(20,10))

plt.subplot(121).set_title("True Values")
plt.scatter(np.arange(len(X_torch)), y_true_index, c='r', marker='o')
# plt.scatter(np.arange(len(X_test_torch)), y_true_index, c='r', marker='o')
plt.xlabel("Picture Number")
plt.ylabel("Category")
plt.yticks((0,1))

plt.subplot(122).set_title("Predicted Values")
plt.scatter(np.arange(len(X_torch)), y_pred_index, c='b', marker='x')
# plt.scatter(np.arange(len(X_test_torch)), y_true_index, c='r', marker='o')
plt.xlabel("Picture Number")
plt.ylabel("Category")
plt.yticks((0,1))
plt.show()

NameError: name 'torch' is not defined

Another interesting way of looking at the results is using Lab 9's _plot_named_tensors(tensor_dict)_. The colorbars below graphically tell us an overall picture of how the classification went for our testing data.

Credit: Andrew Delong, Lab 9, COMP 432 Fall 2020 Concordia University

In [ ]:
def plot_named_tensors(tensor_dict):
    """
    Given a dict of {name: tensor} pairs, plots the tensors side-by-side in a common
    color scale. The name of each tensor is shown above its plot.
    """
    n = len(tensor_dict)
    vmax = max(v.abs().max() for v in tensor_dict.values())
    figsize = (2*n, 6)
    fig, axes = plt.subplots(1, n, figsize=figsize,  constrained_layout=True, squeeze=True)
    axes = axes.flat if isinstance(axes, np.ndarray) else (axes,)
    for (name, v), ax in zip(tensor_dict.items(), axes):
        v = torch.squeeze(v.detach())   # Automatically convert (N,1,D) to (N,D)
        if v.ndim == 1:
            v = v.view(-1, 1)  # Automatically convert (N,) to (N,1)
        assert v.ndim == 2, "couldn't turn tensors[%d] with shape %s into 2D" % (i, v.shape)
        img = ax.matshow(v, vmin=0, vmax=1, cmap=plt.get_cmap('bwr'))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(name)
    fig.colorbar(img, cax=fig.add_axes([0.985, 0.25, 0.03, .5]))   # Add a colorbar on the right 

In [ ]:
plot_named_tensors({'y_pred' : y_pred, 'y_true': y_true})
# plot_named_tensors({'y_pred' : y_pred, 'y_true': y_true_test})

TODO: print accuracies in percentages like in video

Numerically speaking, these are the values we are seeing above:

# Visualizing Weights

TODO: refactor code below into function

For this section we will visualize how each layer in our network is scoring each area of the pictures in our training data set; therefore, as reference, let us show 3 good apples and 3 bad ones from this group of images.

In [ ]:
plt.figure(figsize=(16, 4))


plt.subplot(161)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[0].reshape(390,453,3));
# plt.imshow(X_test_torch[0].reshape(390,453,3));


plt.subplot(162)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[60].reshape(390,453,3));
# plt.imshow(X_test_torch[60].reshape(390,453,3));


plt.subplot(163)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[5].reshape(390,453,3));
# plt.imshow(X_test_torch[5].reshape(390,453,3));


plt.subplot(164)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[57].reshape(390,453,3));
# plt.imshow(X_test_torch[57].reshape(390,453,3));


plt.subplot(165)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[15].reshape(390,453,3));
# plt.imshow(X_test_torch[15].reshape(390,453,3));

plt.subplot(166)
plt.yticks([])
plt.xticks([])
plt.imshow(X_torch[40].reshape(390,453,3));
# plt.imshow(X_test_torch[40].reshape(390,453,3));

Define the _plot_matrix_grid(V)_ to visually analyze a filter's weights.

Credit: Andrew Delong, Lab 9, COMP 432 Fall 2020 Concordia University

In [ ]:
def plot_matrix_grid(V, cmap='bwr'):
    """
    Given an array V containing stacked matrices, plots them in a grid layout.
    V should have shape (K,M,N) where V[k] is a matrix of shape (M,N).
    The default cmap is "bwr" (blue-white-red) but can also be "gray".
    """
    if isinstance(V, torch.Tensor):
        V = V.detach().numpy()
    assert V.ndim == 3, "Expected V to have 3 dimensions, not %d" % V.ndim
    k, m, n = V.shape
    ncol = 8                                     # At most 8 columns
    nrow = min(4, (k + ncol - 1) // ncol)        # At most 4 rows
    V = V[:nrow*ncol]                            # Focus on just the matrices we'll actually plot
    figsize = (2*ncol, max(1, 2*nrow*(m/n)))     # Guess a good figure shape based on ncol, nrow
    fig, axes = plt.subplots(nrow, ncol, sharex=True, sharey=True, figsize=figsize)
    vmax = np.percentile(np.abs(V), [99.9])      # Show the main range of values, between 0.1%-99.9%
    for v, ax in zip(V, axes.flat):
        img = ax.matshow(v, vmin=-vmax, vmax=vmax, cmap=plt.get_cmap(cmap))
        ax.set_xticks([])
        ax.set_yticks([])
    for ax in axes.flat[len(V):]:
        ax.set_axis_off()
    fig.colorbar(img, cax=fig.add_axes([0.92, 0.25, 0.01, .5]))   # Add a colorbar on the right

First layer filter weights:

In [ ]:
W1, b1, W2, b2, W3, b3 = model.parameters()

plot_matrix_grid(W1.reshape(12,5,5), cmap='bwr')

Second layer filter weights:

In [ ]:
plot_matrix_grid(W2.reshape(16,5,5), cmap='bwr')

# How long will my apple last?

Since we have trained, tested and verified the results of our convnet, we can proceed to achieve the goal that was set at the beginning: to determine how long a user's apple has left in the refrigerator or in the pantry.

The first thing we will do is create a map that will match the predicted score of a given apple as a good apple to a number of days. These days represent how long said apple will last in the refrigerator. Later on we will repeat the process but for when storing the apple in the pantry.

Sources for an apple's live before it becomes rotten:
- HERE

In [ ]:
# TODO: create classification map

Now that we have our map, let us load an apple we are interested in knowing how long it has left before going bad.

In [3]:
# TODO: load apple

Next, run this apple though the convnet to obtain a prediction.

In [4]:
# TODO: obtain prediction from neural network

Pass this prediction to the map and obtain a result!

In [ ]:
# TODO: Pass this prediction to the map and obtain a result!